In [11]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_TRACING_v2"] = "true"
os.environ["LANGSMITH_PROJECT"] = "tutorial_chatbot"
os.environ["LANGSMITH_API_KEY"] = ""
os.environ["TAVILY_API_KEY"] = ""

In [12]:
from dotenv import load_dotenv
load_dotenv(dotenv_path="../.env", override=True)

True

In [13]:
from langsmith import utils
utils.tracing_is_enabled()

True

In [14]:
from langchain_ollama import ChatOllama

model = ChatOllama(model=os.getenv("LLM_MODEL"))

In [18]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("What is openBIS and how to use it?")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'url': 'https://www.uzh.ch/blog/hbz/2019/11/20/how-to-use-openbis-as-electronic-lab-notebook/?lang=en', 'content': 'openBIS is a free and open-source data manager developed by the ETH Zurich and supports the full data life cycle from project inception, through data production and analysis, to making the data available to the public. [...] Bauch, A., Adamczyk, I., Buczek, P. et al. openBIS: a flexible framework for managing and analyzing complex data in biology research. BMC Bioinformatics12, 468 (2011)Barillari C,, Ottoz D, Fuentes-Serna, J. et al. openBIS ELN-LIMS: an open-source database for academic laboratories. Bioinformatics, Volume 32, Issue 4, 15 February 2016, Pages 638–640 [...] The openBIS electronic lab notebook, ELN, is a relatively recent addition, and it can replace paper-based lab notebooks. Through a web interface, the ELN is making it easier to capture the sample preparation process, keep track of reagents and protocols, and, importantly, explicitly link the protoco